In [1]:
!pip install pymilvus sentence-transformers torch

In [2]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")


In [3]:
mfd_content = []
with open("./mfd.md", "r") as file:
    for line in file.readlines():
        line = line.strip('\n')
        if len(line)>0:
            mfd_content.append(line)
print(mfd_content[-1],len(mfd_content))

**第五百八十九条** 法律另有规定的，依照其规定。 697


In [4]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

/Users/zhugaojian002/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_mfd.db")
collection_name = "mfd_rag_collection"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

embedding_dim = len(embedding_model.encode_queries("民法典")[0])
print(embedding_dim)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

/Users/zhugaojian002/miniconda3/lib/python3.13/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


768


In [8]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

data = []
model = SentenceTransformer("BAAI/bge-base-zh")
doc_embeddings = model.encode(mfd_content,normalize_embeddings=True)

for i, line in enumerate(tqdm(mfd_content, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 697/697 [00:00<00:00, 4004698.48it/s]


{'insert_count': 697, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

In [9]:
question = "居住权在转让方面有什么规定?"

In [10]:
question

'居住权在转让方面有什么规定?'

In [11]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=model.encode(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [12]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4).encode('utf-8').decode('unicode_escape'))

[
    [
        "##### 第五节 居住权",
        0.8811129927635193
    ],
    [
        "居住权人对居住权的住宅享有占有、使用的权利。",
        0.8690370917320251
    ],
    [
        "**第三百八十四条** 居住权不得转让、继承。",
        0.8651902675628662
    ]
]


In [13]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [14]:
question

'居住权在转让方面有什么规定?'

In [15]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [16]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据《中华人民共和国民法典》第三百八十四条的规定，居住权不得转让。

<translated>
根据《中华人民共和国民法典》第三百八十四条的规定，居住权不得转让。
</translated>


In [17]:
question="宅基地使用权人对宅基地有什么权利？"
search_res = milvus_client.search(
    collection_name=collection_name,
    data=model.encode(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4).encode('utf-8').decode('unicode_escape'))
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

[
    [
        "宅基地使用权人有权在宅基地上建造房屋及其附属设施。",
        0.9068938493728638
    ],
    [
        "##### 第四节 宅基地使用权",
        0.8818694353103638
    ],
    [
        "**第三百七十九条** 宅基地使用权人依法享有宅基地上的房屋及其附属设施的所有权。",
        0.8740971088409424
    ]
]
宅基地使用权人有权在宅基地上建造房屋及其附属设施，并依法享有宅基地上的房屋及其附属设施的所有权。

<translated>
宅基地使用权人有权在宅基地上建造房屋及其附属设施，并依法享有宅基地上的房屋及其附属设施的所有权。
</translated>
